In [ ]:
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")


In [ ]:
import re
def cleanTxt(text):
    text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
    text = re.sub('#', '', text) # Removing '#' hash tag
    text = re.sub('RT[\s]+', '', text) # Removing RT
    text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
    # to remove ',', '.', '!', '?', ';', ':', '(', ')', '[', ']', '{', '}', '-', '_', '*', '/', '\', '|', '~', '`', '+', '=', '<', '>', '^', '&', '$', '#', '@', '©', '®', '™', '°', '•', '…'
    text = re.sub('[^\w\s]','',text)
    text = re.sub('/\S+', '', text) # Removing hyperlink
    text = re.sub('\s+', ' ', text) # Removing multiple spaces
    text = text.lower() # Converting to lowercase
    text = text.strip() # Removing trailing spaces
    text = text.replace('\n', ' ') # Removing newline character
    # to remove stopwords
    text = ' '.join([word for word in text.split() if word not in stpwrd])
    return text

In [ ]:
labels = {
    "LABEL_0": "negative",
    "LABEL_1": "positive",
}

In [ ]:
def get_sentiment_Bert(reviews):
    try:
        reviews = cleanTxt(reviews)
    except:
        pass
    sentiments = sentiment_pipeline(reviews)
    l = sentiments[0]['label']
    s = sentiments[0]['score']
    # return str(l).lower()
    return l

In [ ]:
get_sentiment_Bert("i am a good boy")

In [ ]:
import pandas as pd
data=pd.read_excel("/Users/jatavathpavannaik/Documents/python/Bert/Data_sets/test_data.xlsx")
df=pd.DataFrame(data)

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
def sentiment_scores(sentence):
 
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    try:
        sentiment_dict = sid_obj.polarity_scores(sentence)
    # return sentiment_dict
        if sentiment_dict['compound'] >= 0.1 :
            return "positive",sentiment_dict["compound"]

        elif sentiment_dict['compound'] <= - 0.02 :
            
            return "negative",sentiment_dict['compound']

        else :
            return "neutral",sentiment_dict['compound']
    except Exception as e:
        print({e})

In [ ]:
df['sentiment_vader_score']=df['comments_without_emojis'].apply(lambda x: sentiment_scores(x)[1])
df['sentiment_vader_label']=df['comments_without_emojis'].apply(lambda x: sentiment_scores(x)[0])


In [ ]:
df["sentiment_vader_label"].value_counts()

In [ ]:
positive_df = df[df['sentiment_vader_label'] == 'positive']
negative_df = df[df['sentiment_vader_label'] == 'negative']
neutral_df = df[df['sentiment_vader_label'] == 'neutral']

In [ ]:
positive_df=positive_df.sample(166)
negative_df=negative_df.sample(167)
neutral_df=neutral_df.sample(167)

In [ ]:
conat=pd.concat([positive_df,negative_df,neutral_df])

In [ ]:
conat.info()

In [ ]:
conat['comments_without_emojis'] = conat['comments_without_emojis'].fillna('')

In [ ]:
conat['sentiment_Bert_Model_score']=conat["comments_without_emojis"].astype(str).apply(lambda x: get_sentiment_Bert(x))

In [ ]:
conat['sentiment_Bert_Model_label']=conat["comments_without_emojis"].astype(str).apply(lambda x: get_sentiment_Bert(x))


In [ ]:
conat["sentiment_Bert_Model_label"]=conat["sentiment_Bert_Model_label"].str.lower()

In [ ]:
sid_obj = SentimentIntensityAnalyzer()
sentiment_dict = sid_obj.polarity_scores("i am a good boy")


In [ ]:
sentiment_dict

In [ ]:
sentiment_pipeline("i am good boy")

### my Labels

In [ ]:
# import pandas as pd
# data=pd.read_excel("/Users/jatavathpavannaik/Documents/python/Bert/Data_sets/Bert_Vader.xlsx")
# bv=pd.DataFrame(data)
bv=conat

In [ ]:
bv.columns

In [ ]:
# import numpy as np
# condition = (bv["sentiment_vader_score"] <= 0.5) & (bv["sentiment_vader_score"] >= -0.5)

# bv["label"] = np.where(condition & (bv["sentiment_Bert_Model_score"] >= 0.99),
#                        bv["sentiment_Bert_Model_label"],
#                        "neutral")


# bv.loc[~condition, "label"] = bv["sentiment_Bert_Model_label"]


In [ ]:
import numpy as np
condition = (bv["sentiment_vader_score"] == 0)

bv["label"] = np.where(condition,
                       "neutral",bv["sentiment_Bert_Model_label"])


bv.loc[~condition, "label"] = bv["sentiment_Bert_Model_label"]


In [ ]:
bv["label"].value_counts()

In [ ]:
bv["sentiment_vader_label"].value_counts()

In [ ]:
bv.to_clipboard()

In [ ]:
from sklearn import metrics
Bert_classification_metrics = metrics.classification_report(bv["sentiment_vader_label"], bv["label"])
print("Bert_classification_metrics: \n",Bert_classification_metrics, '\n')
confusion_metrics = metrics.confusion_matrix(bv["sentiment_vader_label"], bv["label"])
print("confusion_metrics: \n",confusion_metrics)

# twitter data

In [ ]:
import pandas as pd
tw=pd.read_csv("/Users/jatavathpavannaik/Documents/python/Bert/English_YT_comments.xlsx")
tw=pd.DataFrame(tw)
tw=tw.sample(500)

In [ ]:
import pandas as pd

# Assuming tw is your DataFrame
# Assuming "category" is the column you want to use for mapping

# Use np.select to create a new column based on conditions
tw['sentiment_label'] = np.select(
    [tw['category'] == 1, tw['category'] == -1, tw['category'] == 0],
    ['positive', 'negative', 'neutral'],
    default='undefined'  # You can choose a default label for other cases
)


In [ ]:
tw

In [ ]:
def get_sentiment_Bert(reviews):
    try:
        reviews = cleanTxt(reviews)
    except:
        pass
    sentiments = sentiment_pipeline(reviews)
    l = sentiments[0]['label']
    s = sentiments[0]['score']
    # return str(l).lower()
    return l

In [ ]:
tw['sentiment_vader_score']=tw['clean_text'].apply(lambda x: sentiment_scores(x)[1])
tw['sentiment_vader_label']=tw['clean_text'].apply(lambda x: sentiment_scores(x)[0])


In [ ]:
tw['sentiment_Bert_Model_score']=tw["clean_text"].astype(str).apply(lambda x: get_sentiment_Bert(x))


In [ ]:
tw['sentiment_Bert_Model_label']=tw["clean_text"].astype(str).apply(lambda x: get_sentiment_Bert(x))


In [ ]:
bv=tw

In [ ]:
import numpy as np
condition = (bv["sentiment_vader_score"] == 0)

bv["label"] = np.where(condition,
                       "neutral",bv["sentiment_Bert_Model_label"])


bv.loc[~condition, "label"] = bv["sentiment_Bert_Model_label"]

In [ ]:
bv["sentiment_Bert_Model_label"]=bv["sentiment_Bert_Model_label"].str.lower()
bv["label"]=bv["label"].str.lower()

In [ ]:
bv

In [ ]:
from sklearn import metrics
Bert_classification_metrics = metrics.classification_report(bv["sentiment_label"], bv["label"])
print("Bert_classification_metrics: \n",Bert_classification_metrics, '\n')
confusion_metrics = metrics.confusion_matrix(bv["sentiment_label"], bv["label"])
print("confusion_metrics: \n",confusion_metrics)

In [ ]:
bv

#### youtube ysrcp

In [1]:
import pandas as pd
data=pd.read_excel("/Users/jatavathpavannaik/Documents/python/Bert/Phase_2_Translated_text_Youtube_hashtag_comments_24Feb_25Feb.xlsx")
df=pd.DataFrame(data)

In [28]:
# df=df[df["consol_lang_tag"]=="en"]
# df.rename(columns={"Preprocessed_text":"translated_text"},inplace=True)

In [3]:
df.dropna(subset=["translated_text"],inplace=True)

In [5]:
# df.drop(columns="translated_text",inplace=True)

In [7]:
# df.rename(columns={"Preprocessed_text":"translated_text"},inplace=True)

In [4]:
df

,Unnamed: 0,_id,author_channel_id,video_id,id1,comment,publish_date,party,upload_date,Html_tags_removed,...,language,accuracy,lang,translated_text,lower_comment,Punctuation_removed,whitespace_removed,stop_words_removed,preprocessed_text,sentiment_vader
0,0,65dd95978ede59313b460851,UCiHhoalS6Styu-HunXIOQpw,rRRSEMy2Cyo,UgzQHlyc4zGVMhN1ASt4AaABAg,అడుడం ఆంధ్ర పుణ్యమా అంటూ విజయనగరం నుండి ఒక సామ...,2024-02-25 06:35:58,TDP,2024-02-27 01:24:52,అడుడం ఆంధ్ర పుణ్యమా అంటూ విజయనగరం నుండి ఒక సామ...,...,__label__te,1.00,te,Dhoni's Chance in CSK Team from Vijayanagaram,dhoni's chance in csk team from vijayanagaram,dhonis chance in csk team from vijayanagaram,dhonis chance in csk team from vijayanagaram,dhonis chance csk team vijayanagaram,dhonis chance csk team vijayanagaram,Positive
1,1,65dd95978ede59313b460852,UCeL4EY4a2lAJMu9LguQxaMQ,rRRSEMy2Cyo,UgzLpooj1SvITHfHe914AaABAg,commentary 🤣🤣🤣🤣🤣,2024-02-25 05:55:51,TDP,2024-02-27 01:24:52,commentary 🤣🤣🤣🤣🤣,...,__label__en,0.82,en,commentary,commentary,commentary,commentary,commentary,commentary,Neutral
2,2,65dd95978ede59313b460853,UCL6aHHP-1098sMmqSvnGcNw,rRRSEMy2Cyo,UgyBZ_37volR5Z3CexR4AaABAg,Adudaam Andhra lo selected persons ke chance i...,2024-02-25 02:15:35,TDP,2024-02-27 01:24:52,Adudaam Andhra lo selected persons ke chance i...,...,__label__en,0.43,en,Adudaam Andhra lo selected persons ke chance i...,adudaam andhra lo selected persons ke chance i...,adudaam andhra lo selected persons ke chance i...,adudaam andhra lo selected persons ke chance i...,adudaam andhra lo selected persons ke chance i...,adudaam andhra lo selected person ke chance ic...,Positive
3,3,65dd95978ede59313b460ab1,UCmf0Ti8HvbKxMxNqG6wc4FA,-DzXX8ZUH54,Ugzq9QRQUlKL840ltiZ4AaABAg,E channel ki andharu report kottanddi,2024-02-24 08:36:25,TDP,2024-02-27 01:24:52,E channel ki andharu report kottanddi,...,__label__en,0.73,en,E channel ki andharu report kottanddi,e channel ki andharu report kottanddi,e channel ki andharu report kottanddi,e channel ki andharu report kottanddi,e channel ki andharu report kottanddi,e channel ki andharu report kottanddi,Neutral
4,4,65dd95978ede59313b460ab2,UCmf0Ti8HvbKxMxNqG6wc4FA,-DzXX8ZUH54,UgyOfU5micx2P3BjQVp4AaABAg,Bro andhari ki chepthuna idhi fake news .jagan...,2024-02-24 08:34:42,TDP,2024-02-27 01:24:52,Bro andhari ki chepthuna idhi fake news .jagan...,...,__label__en,0.43,en,Bro andhari ki chepthuna idhi fake news .jagan...,bro andhari ki chepthuna idhi fake news .jagan...,bro andhari ki chepthuna idhi fake news jagan ...,bro andhari ki chepthuna idhi fake news jagan ...,bro andhari ki chepthuna idhi fake news jagan ...,bro andhari ki chepthuna idhi fake news jagan ...,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17898,18040,65e07d16d3e0d87287fe7060,UC6fXXdIVR-zw03EC_NF6CZQ,T2M4NEAuAeE,UgyHc3ms9ycwuLxUhfV4AaABAg,Eppudu eddaru kalisi vocharu next inka entha m...,2024-02-24 10:28:40,TDP,2024-02-29 06:18:13,Eppudu eddaru kalisi vocharu next inka entha m...,...,__label__en,0.22,en,Eppudu eddaru kalisi vocharu next inka entha m...,eppudu eddaru kalisi vocharu next inka entha m...,eppudu eddaru kalisi vocharu next inka entha m...,eppudu eddaru kalisi vocharu next inka entha m...,eppudu eddaru kalisi vocharu next inka entha m...,eppudu eddaru kalisi vocharu next inka entha m...,Neutral
17899,18041,65e07d16d3e0d87287fe7061,UCgj1IMG5noI1GjDdDq_O_GQ,T2M4NEAuAeE,UgwNqONf4PMObj2129t4AaABAg,Orey mundu aa 24 seats gelipinchandira chalu,2024-02-24 10:28:26,TDP,2024-02-29 06:18:13,Orey mundu aa 24 seats gelipinchandira chalu,...,__label__en,0.60,en,Orey mundu aa seats gelipinchandira chalu,orey mundu aa seats gelipinchandira chalu,orey mundu aa seats gelipinchandira chalu,orey mundu aa seats gelipinchandira chalu,orey mundu aa seats gelipinchandira chalu,orey mundu aa seat gelipinchandira chalu,Neutral
17900,18042,65e07d16d3e0d87287fe7062,UCrn4a7lHQ1BpEy4FxMadWUA,T2M4NEAuAeE,UgwTNJ9t7vpCXUf5ztt4AaABAg,Na mogga nachite pettukoava,2024-02-24 10:27:36,TDP,202

In [5]:
df["translated_text"]=df["translated_text"].str.lower().astype(str)

In [6]:
labels = {
    "LABEL_0": "negative",
    "LABEL_1": "positive",
}

In [7]:
import re
def cleanTxt(text):
    text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
    text = re.sub('#', '', text) # Removing '#' hash tag
    text = re.sub('$', '', text) # Removing '#' hash tag
    text = re.sub('RT[\s]+', '', text) # Removing RT
    text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
    # to remove ',', '.', '!', '?', ';', ':', '(', ')', '[', ']', '{', '}', '-', '_', '*', '/', '\', '|', '~', '`', '+', '=', '<', '>', '^', '&', '$', '#', '@', '©', '®', '™', '°', '•', '…'
    text = re.sub('[^\w\s]','',text)
    text = re.sub('/\S+', '', text) # Removing hyperlink
    text = re.sub('\s+', ' ', text) # Removing multiple spaces
    text = text.lower() # Converting to lowercase
    text = text.strip() # Removing trailing spaces
    text = text.replace('\n', ' ') # Removing newline character
    # to remove stopwords
    # text = ' '.join([word for word in text.split() if word not in stpwrd])
    return text

In [8]:
def get_sentiment_Bert(reviews):
    try:
        reviews = cleanTxt(reviews)
    except:
        pass
    sentiments = sentiment_pipeline(reviews)
    l = sentiments[0]['label']
    s = sentiments[0]['score']
    # return str(l).lower()
    return l,s

In [9]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
def sentiment_scores(sentence):
 
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    try:
        sentiment_dict = sid_obj.polarity_scores(sentence)
    # return sentiment_dict
        if sentiment_dict['compound'] >= 0.1 :
            return "positive",sentiment_dict["compound"]

        elif sentiment_dict['compound'] <= - 0.02 :
            
            return "negative",sentiment_dict['compound']

        else :
            return "neutral",sentiment_dict['compound']
    except Exception as e:
        print({e})

In [10]:
df[['sentiment_vader_label', 'sentiment_vader_score']] = df['translated_text'].apply(sentiment_scores).apply(pd.Series)

In [11]:
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

/Users/jatavathpavannaik/anaconda3/envs/Bert/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-29 18:47:26.357025: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-02-29 18:47:26.357060: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-02-29 18:47:26.357070: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-02-29 18:47:26.357142: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-29 18:47:26.357187: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with

In [12]:
df[['sentiment_Bert_label', 'sentiment_Bert_score']] = df['translated_text'].apply(get_sentiment_Bert).apply(pd.Series)

Token indices sequence length is longer than the specified maximum sequence length for this model (526 > 512). Running this sequence through the model will result in indexing errors


In [13]:
df["sentiment_Bert_label"]=df["sentiment_Bert_label"].str.lower()

In [14]:
import numpy as np
condition = ((df["sentiment_Bert_score"] <0.99)&(df["sentiment_vader_score"]<=0.09)&(df["sentiment_vader_score"]>=-0.01))

df["label"] = np.where(condition,
                       "neutral",df["sentiment_Bert_label"])

df.loc[~condition, "label"] = df["sentiment_Bert_label"]

In [15]:
df["label"].value_counts()

label
neutral     7366
negative    5448
positive    5089
Name: count, dtype: int64

In [20]:
df.to_clipboard()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
df.to_excel("TDP_sentiment_comments_Hashtags_24Feb_25Feb_2024.xlsx")

In [ ]:
#range(-0.0191,0.0972)